In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data Loading

In [3]:
df=pd.read_excel("data/지체장애_고혈압_결측치 대체_데이터.xlsx")
df.head()

,id,생년,장애등록 연도,본인을 포함한 총 가구원수,본인을 포함한 총 장애인수,월 평균 총가구소득,가구 월평균 지출액,장애발생시 연령,월 혈압약 일수,관절통증 정도,...,인공호흡기-소지,인공호흡기-사용,사용빈도,1일 이용시간,이용경험-상담과 평가,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,인터넷 사용 여부,01)만성질환명(고혈압)
0,1,1970,2008,3,1,280,250,38,0,0,...,2,0,0,0,2,2,1,1,1,0
1,2,1949,2010,3,1,180,150,20,30,0,...,2,0,0,0,2,1,2,2,2,1
2,4,1958,2011,2,1,350,200,51,0,0,...,2,0,0,0,2,2,1,2,2,0
3,5,1955,2009,3,2,154,154,53,30,0,...,2,0,1,3,2,2,1,2,2,1
4,6,1936,2010,2,1,275,270,62,0,7,...,2,0,2,9,1,1,2,2,2,0


In [4]:
for columns in df.columns : #혈압과 관련된 칼럼 추출 
    if "혈압" in columns:
        print(columns)

월 혈압약 일수
혈압약 복용 여부
01)만성질환명(고혈압)


In [5]:
df2=df.drop({'월 혈압약 일수','혈압약 복용 여부'}, axis=1)

In [6]:
df2.head()

,id,생년,장애등록 연도,본인을 포함한 총 가구원수,본인을 포함한 총 장애인수,월 평균 총가구소득,가구 월평균 지출액,장애발생시 연령,관절통증 정도,운동 시간(분),...,인공호흡기-소지,인공호흡기-사용,사용빈도,1일 이용시간,이용경험-상담과 평가,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,인터넷 사용 여부,01)만성질환명(고혈압)
0,1,1970,2008,3,1,280,250,38,0,90,...,2,0,0,0,2,2,1,1,1,0
1,2,1949,2010,3,1,180,150,20,0,0,...,2,0,0,0,2,1,2,2,2,1
2,4,1958,2011,2,1,350,200,51,0,60,...,2,0,0,0,2,2,1,2,2,0
3,5,1955,2009,3,2,154,154,53,0,60,...,2,0,1,3,2,2,1,2,2,1
4,6,1936,2010,2,1,275,270,62,7,0,...,2,0,2,9,1,1,2,2,2,0


### 연속형 변수 스케일링(Min-Max Scaling)

In [56]:
print(list(df2.columns))

['id', '생년', '장애등록 연도', '본인을 포함한 총 가구원수', '본인을 포함한 총 장애인수', '월 평균 총가구소득', '가구 월평균 지출액', '장애발생시 연령', '관절통증 정도', '운동 시간(분)', '키(센티)', '몸무게(kg)', '지난 2주간 아팠던 날수', '성별', '장애등록 여부', '장애등급', '가구 유형', '가구 주된 수입원', '주택 형태', '장애부위(상지)', '장애부위(하지)', '장애부위(척추)', '가장 불편한부위', '장애발생시기', '장애주된 원인', '질병명', '산업재해인정여부', '주된 진단명', '건강보험가입여부 및 형태', '국가유공자, 차상위경감 대상여부', '현재 지속적 진료여부', '주관적 건강 평가', '건강검진 여부', '암검진 여부', '구강건강 검진 여부', '인플루엔자 예방접종여부', '입원치료 여부', '외래진료 여부(치과,보건소,한의원등 포함)', '병의원이나 응급실 이용 사고나중독 발생 여부', '사고발생내용1', '사고발생내용2', '사고발생내용3', '미충족의료 경험 유무', '희귀난치성질환 여부', '희귀난치성질환 등록 여부', '혈당관리 치료 여부', '혈당치료 - 인슐린주사', '혈당치료 - 당뇨병약', '혈당치료 - 비약물요법', '혈당치료 - 기타', '우울증 치료 여부', '암 종류1', '암 종류2', '암 종류3', '지난 2주간 아팠음 여부', '물리치료-이용여부', '작업치료-이용여부', '언어치료-이용여부', '음악치료-이용여부', '놀이치료-이용여부', '미술치료-이용여부', '심리행동치료-이용여부', '기타-이용여부', '흡연 여부', '최근 1년간 금연시도', '흡연예방교육 여부', '음주횟수', '평균음주량', '건광관리운동', '운동 주기', '재활운동체육 필요여부', '장애인 체육지도자 필요여부', '일정한 시간에 식사여부', '고른 식품 섭취여부', '음식물 씹는데 불편감 여부', '본인 체형 평가', '일상생활 중 스트레스정도', '슬프거나 

In [7]:
continu_df = df2[['id', '생년', '장애등록 연도', '본인을 포함한 총 가구원수', '본인을 포함한 총 장애인수', '월 평균 총가구소득', '가구 월평균 지출액', 
                  '장애발생시 연령', '관절통증 정도', '운동 시간(분)', '키(센티)', '몸무게(kg)', '지난 2주간 아팠던 날수']]
continu_df.head(5)

,id,생년,장애등록 연도,본인을 포함한 총 가구원수,본인을 포함한 총 장애인수,월 평균 총가구소득,가구 월평균 지출액,장애발생시 연령,관절통증 정도,운동 시간(분),키(센티),몸무게(kg),지난 2주간 아팠던 날수
0,1,1970,2008,3,1,280,250,38,0,90,178.0,77.0,14
1,2,1949,2010,3,1,180,150,20,0,0,163.0,55.0,14
2,4,1958,2011,2,1,350,200,51,0,60,158.0,63.0,14
3,5,1955,2009,3,2,154,154,53,0,60,163.0,62.0,14
4,6,1936,2010,2,1,275,270,62,7,0,154.0,57.0,14


In [8]:
mm_scaler = MinMaxScaler()
continu_mm = mm_scaler.fit_transform(continu_df.iloc[:, 1:])
continu_mm_df = pd.DataFrame(continu_mm, columns=continu_df.columns[1:])
continu_mm_df = pd.concat([continu_df.id, continu_mm_df], axis = 1)
continu_mm_df.head()

,id,생년,장애등록 연도,본인을 포함한 총 가구원수,본인을 포함한 총 장애인수,월 평균 총가구소득,가구 월평균 지출액,장애발생시 연령,관절통증 정도,운동 시간(분),키(센티),몸무게(kg),지난 2주간 아팠던 날수
0,1,0.604651,0.689655,0.250,0.000000,0.093333,0.250,0.417582,0.0,0.250000,0.733945,0.378571,1.0
1,2,0.360465,0.758621,0.250,0.000000,0.060000,0.150,0.219780,0.0,0.000000,0.596330,0.221429,1.0
2,4,0.465116,0.793103,0.125,0.000000,0.116667,0.200,0.560440,0.0,0.166667,0.550459,0.278571,1.0
3,5,0.430233,0.724138,0.250,0.333333,0.051333,0.154,0.582418,0.0,0.166667,0.596330,0.271429,1.0
4,6,0.209302,0.758621,0.125,0.000000,0.091667,0.270,0.681319,0.7,0.000000,0.513761,0.235714,1.0


### 범주형 변수 스케일링(One-Hot Encoder)

In [9]:
catego_df = df2.loc[:, '성별':'인터넷 사용 여부']
catego_df = catego_df.astype(dtype='object')
catego_df.head()

,성별,장애등록 여부,장애등급,가구 유형,가구 주된 수입원,주택 형태,장애부위(상지),장애부위(하지),장애부위(척추),가장 불편한부위,...,인공호흡기-필요,인공호흡기-소지,인공호흡기-사용,사용빈도,1일 이용시간,이용경험-상담과 평가,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,인터넷 사용 여부
0,1,1,6,21,1,2,0,1,0,36,...,2,2,0,0,0,2,2,1,1,1
1,1,1,6,21,3,2,1,0,0,14,...,2,2,0,0,0,2,1,2,2,2
2,2,1,5,11,1,2,0,0,1,38,...,2,2,0,0,0,2,2,1,2,2
3,2,1,5,29,1,2,0,0,1,38,...,2,2,0,1,3,2,2,1,2,2
4,2,1,3,24,3,2,0,3,0,36,...,2,2,0,2,9,1,1,2,2,2


In [10]:
catego_onehot_df = pd.get_dummies(catego_df)
catego_onehot_df = pd.concat([continu_df.id, catego_onehot_df], axis = 1)
catego_onehot_df.head()

,id,성별_1,성별_2,장애등록 여부_1,장애등급_1,장애등급_2,장애등급_3,장애등급_4,장애등급_5,장애등급_6,...,이용경험-상담과 평가_1,이용경험-상담과 평가_2,휴대폰 사용 여부_1,휴대폰 사용 여부_2,스마트폰 사용 여부_1,스마트폰 사용 여부_2,컴퓨터 사용 여부_1,컴퓨터 사용 여부_2,인터넷 사용 여부_1,인터넷 사용 여부_2
0,1,1,0,1,0,0,0,0,0,1,...,0,1,0,1,1,0,1,0,1,0
1,2,1,0,1,0,0,0,0,0,1,...,0,1,1,0,0,1,0,1,0,1
2,4,0,1,1,0,0,0,0,1,0,...,0,1,0,1,1,0,0,1,0,1
3,5,0,1,1,0,0,0,0,1,0,...,0,1,0,1,1,0,0,1,0,1
4,6,0,1,1,0,0,1,0,0,0,...,1,0,1,0,0,1,0,1,0,1


### 범주형 변수 + 연속형 변수 + y 변수 결합

In [11]:
# y 변수
y = df2[['id', '01)만성질환명(고혈압)']]

In [12]:
# merge
first_merge = pd.merge(continu_mm_df, catego_onehot_df, on = 'id')
new_merge = pd.merge(first_merge, y, on = 'id').drop(columns='id')
new_merge.head()

,생년,장애등록 연도,본인을 포함한 총 가구원수,본인을 포함한 총 장애인수,월 평균 총가구소득,가구 월평균 지출액,장애발생시 연령,관절통증 정도,운동 시간(분),키(센티),...,이용경험-상담과 평가_2,휴대폰 사용 여부_1,휴대폰 사용 여부_2,스마트폰 사용 여부_1,스마트폰 사용 여부_2,컴퓨터 사용 여부_1,컴퓨터 사용 여부_2,인터넷 사용 여부_1,인터넷 사용 여부_2,01)만성질환명(고혈압)
0,0.604651,0.689655,0.250,0.000000,0.093333,0.250,0.417582,0.0,0.250000,0.733945,...,1,0,1,1,0,1,0,1,0,0
1,0.360465,0.758621,0.250,0.000000,0.060000,0.150,0.219780,0.0,0.000000,0.596330,...,1,1,0,0,1,0,1,0,1,1
2,0.465116,0.793103,0.125,0.000000,0.116667,0.200,0.560440,0.0,0.166667,0.550459,...,1,0,1,1,0,0,1,0,1,0
3,0.430233,0.724138,0.250,0.333333,0.051333,0.154,0.582418,0.0,0.166667,0.596330,...,1,0,1,1,0,0,1,0,1,1
4,0.209302,0.758621,0.125,0.000000,0.091667,0.270,0.681319,0.7,0.000000,0.513761,...,0,1,0,0,1,0,1,0,1,0


In [13]:
x_data=new_merge[new_merge.columns[:-1]]
x_data.head()

,생년,장애등록 연도,본인을 포함한 총 가구원수,본인을 포함한 총 장애인수,월 평균 총가구소득,가구 월평균 지출액,장애발생시 연령,관절통증 정도,운동 시간(분),키(센티),...,이용경험-상담과 평가_1,이용경험-상담과 평가_2,휴대폰 사용 여부_1,휴대폰 사용 여부_2,스마트폰 사용 여부_1,스마트폰 사용 여부_2,컴퓨터 사용 여부_1,컴퓨터 사용 여부_2,인터넷 사용 여부_1,인터넷 사용 여부_2
0,0.604651,0.689655,0.250,0.000000,0.093333,0.250,0.417582,0.0,0.250000,0.733945,...,0,1,0,1,1,0,1,0,1,0
1,0.360465,0.758621,0.250,0.000000,0.060000,0.150,0.219780,0.0,0.000000,0.596330,...,0,1,1,0,0,1,0,1,0,1
2,0.465116,0.793103,0.125,0.000000,0.116667,0.200,0.560440,0.0,0.166667,0.550459,...,0,1,0,1,1,0,0,1,0,1
3,0.430233,0.724138,0.250,0.333333,0.051333,0.154,0.582418,0.0,0.166667,0.596330,...,0,1,0,1,1,0,0,1,0,1
4,0.209302,0.758621,0.125,0.000000,0.091667,0.270,0.681319,0.7,0.000000,0.513761,...,1,0,1,0,0,1,0,1,0,1


In [14]:
y_data=new_merge[new_merge.columns[-1:]]
y_data.head()

,01)만성질환명(고혈압)
0,0
1,1
2,0
3,1
4,0


In [72]:
y_data.value_counts()

01)만성질환명(고혈압)
1                1528
0                1158
dtype: int64

### Lasso

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [16]:
# lasso에 사용될 alpha 파라미터의 값들을 정의
alphas = [0.1, 0.01, 0.001, 0.0001]

# 각 alpha에 따른 회귀 계수 값을 데이터로 저장하기 위한 DataFrame 생성  
coeff_df = pd.DataFrame()

# alphas list 값을 iteration하면서 alpha에 따른 평균 rmse 구하고, 회귀 계수 값 시각화 및 데이터 저장. pos는 axis의 위치 지정
for pos, alpha in enumerate(alphas):
    lasso = Lasso(alpha = alpha)
    lasso.fit(x_data, y_data)
   
    #cross_val_score를 이용하여 5 fold의 평균 RMSE 계산
    neg_mse_scores = cross_val_score(lasso, x_data, y_data, scoring="neg_mean_squared_error", cv = 5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    print('alpha {0} 일 때 5 folds 의 평균 RMSE : {1:.3f} '.format(alpha,avg_rmse))
    print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))
    
     # alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 컬럼으로 추가.  
    coeff = pd.Series(data=lasso.coef_, index=x_data.columns )
    colname='alpha:'+str(alpha)
    coeff_df[colname] = coeff
    # 막대 그래프로 각 alpha 값에서의 회귀 계수를 시각화. 회귀 계수값이 높은 순으로 표현
    coeff = coeff.sort_values(ascending=False)

alpha 0.1 일 때 5 folds 의 평균 RMSE : 0.496 
사용한 특성의 수 : 0
alpha 0.01 일 때 5 folds 의 평균 RMSE : 0.461 
사용한 특성의 수 : 22
alpha 0.001 일 때 5 folds 의 평균 RMSE : 0.457 
사용한 특성의 수 : 190


C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8695673838652169, tolerance: 0.05264281060958581
  model = cd_fast.enet_coordinate_descent(
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7886302404722301, tolerance: 0.05238808748255009
  model = cd_fast.enet_coordinate_descent(


alpha 0.0001 일 때 5 folds 의 평균 RMSE : 0.497 
사용한 특성의 수 : 517


In [17]:
coeff_df[coeff_df['alpha:0.01']!=0].index

Index(['지난 2주간 아팠던 날수', '장애등급_4', '가구 유형_21', '가장 불편한부위_36', '주된 진단명_6',
       '현재 지속적 진료여부_1', '현재 지속적 진료여부_2', '인플루엔자 예방접종여부_1', '인플루엔자 예방접종여부_2',
       '혈당관리 치료 여부_0', '본인 체형 평가_4', '2)이용_만성질환관리_1', '3)이용_장애관리_재활_2',
       '지팡이-필요_1', '지팡이-필요_2', '지팡이-소지_1', '휴대폰 사용 여부_1', '휴대폰 사용 여부_2',
       '스마트폰 사용 여부_1', '스마트폰 사용 여부_2', '컴퓨터 사용 여부_1', '컴퓨터 사용 여부_2'],
      dtype='object')

In [18]:
coeff_df[coeff_df['alpha:0.01']!=0]

,alpha:0.1,alpha:0.01,alpha:0.001,alpha:0.0001
지난 2주간 아팠던 날수,0.0,3.878474e-01,5.342239e-01,6.573344e-01
장애등급_4,0.0,1.458301e-02,2.037661e-02,1.540804e-02
가구 유형_21,-0.0,-4.664838e-02,-6.799369e-02,-6.328075e-02
가장 불편한부위_36,0.0,6.477544e-03,0.000000e+00,1.528439e-02
주된 진단명_6,0.0,3.318336e-03,2.354283e-02,3.644749e-03
현재 지속적 진료여부_1,0.0,1.368845e-01,2.455252e-01,2.602305e-01
현재 지속적 진료여부_2,-0.0,-2.905718e-16,-0.000000e+00,-4.829105e-17
인플루엔자 예방접종여부_1,0.0,1.678343e-02,9.524378e-03,2.056566e-01
인플루엔자 예방접종여부_2,-0.0,-1.448217e-17,-0.000000e+00,-1.208740e-14
혈당관리 치료 여부_0,-0.0,-3.154111e-02,-3.231597e-02,-2.607031e-02


In [81]:
plt.rc('font', family = 'Gulim')
plt.figure(figsize=(10, 150))
sns.heatmap(coeff_df.sort_values(by=sort_column, ascending=False), annot=True, linewidths=.5, cmap='seismic')